In [6]:
# %cd PaddleOCR/
!python PaddleOCR/infer_rec.py -c PaddleOCR/configs/rec/PP-OCRv3/multi_language/latin_PP-OCRv3_rec.yml \
    -o Global.checkpoints=PaddleOCR/output/v3_latin_mobile/best_model/model \
    Global.infer_img=PaddleOCR/ocr_datasets/test/20230119__SFC__Bao_cao_tai_chinh_quy_1_nien_do_20222023_Signed_page_2_crop_60.jpg
# %cd ..

[2024/07/15 10:55:56] ppocr INFO: Architecture : 
[2024/07/15 10:55:56] ppocr INFO:     Backbone : 
[2024/07/15 10:55:56] ppocr INFO:         last_conv_stride : [1, 2]
[2024/07/15 10:55:56] ppocr INFO:         last_pool_kernel_size : [2, 2]
[2024/07/15 10:55:56] ppocr INFO:         last_pool_type : avg
[2024/07/15 10:55:56] ppocr INFO:         name : MobileNetV1Enhance
[2024/07/15 10:55:56] ppocr INFO:         scale : 0.5
[2024/07/15 10:55:56] ppocr INFO:     Head : 
[2024/07/15 10:55:56] ppocr INFO:         head_list : 
[2024/07/15 10:55:56] ppocr INFO:             CTCHead : 
[2024/07/15 10:55:56] ppocr INFO:                 Head : 
[2024/07/15 10:55:56] ppocr INFO:                     fc_decay : 1e-05
[2024/07/15 10:55:56] ppocr INFO:                 Neck : 
[2024/07/15 10:55:56] ppocr INFO:                     depth : 2
[2024/07/15 10:55:56] ppocr INFO:                     dims : 64
[2024/07/15 10:55:56] ppocr INFO:                     hidden_dims : 120
[2024/07/15 10:55:56] ppocr I

In [2]:
from paddleocr import PaddleOCR

paddle_ocr = PaddleOCR(
    use_angle_cls=True,
    # lang="vi",
    rec_model_dir="/home/nguyen/workplace/signboard_ocr/inference/latin_PP-OCRv3_rec",
    rec_char_dict_path="paddleocr_data/vietnamese_dict.txt",
)  # need to run only once to download and load model into memory

[2024/07/29 09:15:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/nguyen/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/nguyen/workplace/signboard_ocr/inference/latin_PP-OCRv3_rec', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, ma

In [3]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

path = "generated_text_recognition_dataset/undistorted_images"
image_names = np.array(sorted(os.listdir(path)))
# n_images = len(image_names)
# batch_size = 1
# n_batches = int(np.ceil(n_images / batch_size))


def load_images(batch_names):
    images = []
    for name in batch_names:
        img = np.array(Image.open(os.path.join(path, name)))
        images.append(img)
    if len(batch_names) == 1:
        return img
    return images


count = 0
for name in tqdm(image_names):
    # batch_names = image_names[batch_size * i : min(batch_size * (i + 1), n_images)]
    # images = load_images(batch_names)
    image = np.array(Image.open(os.path.join(path, name)))
    result = paddle_ocr.ocr(image, cls=True, rec=False)[0]
    if result is None:
        with open("cant_detect.txt", "a") as f:
            f.write(f"{name}\n")
    # else:
    #     bbox = result[0]
    # if count == 10:
    #     break
    # count += 1

  0%|          | 0/17105 [00:00<?, ?it/s]

100%|██████████| 17105/17105 [21:42<00:00, 13.13it/s] 


In [5]:
import shutil

with open("generated_text_recognition_dataset/undetectable_labels.txt", "r") as f:
    lines = f.readlines()
for line in lines:
    shutil.move(
        os.path.join(
            "generated_text_recognition_dataset/undetectable_images", line[:-1]
        ),
        "generated_text_recognition_dataset/undistorted_images",
    )

In [ ]:
with open('generated_text_recognition_dataset/undetectable_labels.txt', 'r') as f:
    undetectable = f.readlines()
undetectable = [line[:-1] for line in undetectable]



In [10]:
%cd PaddleOCR/
!python3 tools/infer/predict_rec.py \
    --image_dir="ocr_datasets/test/20230119__SFC__Bao_cao_tai_chinh_quy_1_nien_do_20222023_Signed_page_2_crop_60.jpg" \
    --rec_model_dir="inference/latin_PP-OCRv3_rec" \
    --rec_char_dict_path="ppocr/utils/dict/vietnamese_dict.txt" \
    --vis_font_path="doc/fonts/korean.ttf" \
    --use_gpu=False
%cd ..

/home/nguyen/workplace/signboard_ocr/PaddleOCR


--------------------------------------
C++ Traceback (most recent call last):
--------------------------------------
0   paddle_infer::Predictor::Predictor(paddle::AnalysisConfig const&)
1   std::unique_ptr<paddle::PaddlePredictor, std::default_delete<paddle::PaddlePredictor> > paddle::CreatePaddlePredictor<paddle::AnalysisConfig, (paddle::PaddleEngineKind)2>(paddle::AnalysisConfig const&)
2   paddle::AnalysisPredictor::Init(std::shared_ptr<paddle::framework::Scope> const&, std::shared_ptr<paddle::framework::ProgramDesc> const&)
3   paddle::AnalysisPredictor::PrepareProgram(std::shared_ptr<paddle::framework::ProgramDesc> const&)
4   paddle::AnalysisPredictor::OptimizeInferenceProgram()
5   paddle::inference::analysis::Analyzer::RunAnalysis(paddle::inference::analysis::Argument*)
6   paddle::inference::analysis::IrAnalysisPass::RunImpl(paddle::inference::analysis::Argument*)
7   paddle::inference::analysis::IRPassManager::Apply(std::uniqu

image orientation + layout analysis + table recognition

In [1]:
import os
import cv2
from paddleocr import PPStructure, draw_structure_result, save_structure_res

table_engine = PPStructure(show_log=True, image_orientation=True)

save_folder = "./output"
img_path = "PaddleOCR/ppstructure/docs/table/1.png"
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split(".")[0])

for line in result:
    line.pop("img")
    print(line)

from PIL import Image

font_path = "PaddleOCR/doc/fonts/simfang.ttf"  # PaddleOCR下提供字体包
image = Image.open(img_path).convert("RGB")
im_show = draw_structure_result(image, result, font_path=font_path)
im_show = Image.fromarray(im_show)
im_show.save("result.jpg")

[2024/07/17 08:31:44] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/nguyen/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/nguyen/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le

100%|██████████| 7.40M/7.40M [00:12<00:00, 583kiB/s]

[2024/07/17 08:31:58] ppcls WARNING: The current running environment does not support the use of GPU. CPU has been used instead.


[2024/07/17 08:32:02] ppocr DEBUG: dt_boxes num : 230, elapsed : 1.0707526206970215
[2024/07/17 08:33:25] ppocr DEBUG: rec_res num  : 230, elapsed : 82.87537503242493
[2024/07/17 08:33:27] ppocr DEBUG: dt_boxes num : 108, elapse : 0.38725900650024414
[2024/07/17 08:34:07] ppocr DEBUG: rec_res num  : 108, elapse : 39.62550210952759
[2024/07/17 08:34:09] ppocr DEBUG: dt_boxes num : 77, elapse : 0.36330699920654297
[2024/07/17 08:34:36] ppocr DEBUG: rec_res num  : 77, elapse : 26.43921661376953
{'type': 'text', 'bbox': [460, 0, 856, 117], 'res': [], 'img_idx': 0, 'score': 0.8404665589332581}
{'type': 'text', 'bbox': [30, 0, 425, 102], 'res': [], 'img_idx': 0, 'score': 0.7354930639266968}
{'type': 'title', 'bbox': [308, 128, 424, 143], 'res': [], 'img_idx': 0, 'score': 0.9530608057975769}
{'type': 'figure', 'bbox': [21, 547, 860, 847], 'res': [{'text': 'M (P)', 'confidence': 0.6822103261947632, 'text_region': [[94.0, 540.0], [173.0, 540.0], [173.0, 554.0], [94.0, 554.0]]}, {'text': '0M ()'

OSError: cannot open resource

layout analysis

In [2]:
import os
import cv2
from paddleocr import PPStructure, save_structure_res

table_engine = PPStructure(table=False, ocr=False, show_log=True)

save_folder = "./output"
img_path = "PaddleOCR/ppstructure/docs/table/1.png"
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, save_folder, os.path.basename(img_path).split(".")[0])

for line in result:
    line.pop("img")
    print(line)

[2024/07/17 08:45:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/nguyen/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/nguyen/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le

In [5]:
import os
import cv2
from paddleocr import PPStructure, save_structure_res

ocr_engine = PPStructure(table=False, ocr=True, show_log=True)

save_folder = "./output"
img_path = "PaddleOCR/ppstructure/docs/recovery/UnrealText.pdf"
result = ocr_engine(img_path)
for index, res in enumerate(result):
    save_structure_res(
        res, save_folder, os.path.basename(img_path).split(".")[0], index
    )

for res in result:
    for line in res:
        line.pop("img")
        print(line)

[2024/07/17 08:47:57] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/nguyen/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/nguyen/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le

In [4]:
%cd PaddleOCR/
!python3 tools/infer_kie_token_ser.py \
    -c configs/kie/vi_layoutxlm/ser_vi_layoutxlm_xfund_zh.yml \
    -o Architecture.Backbone.checkpoints=../paddleocr_data/pretrained_model/ser_vi_layoutxlm_xfund_pretrained/best_accuracy \
    Global.infer_img=./ppstructure/docs/kie/input/zh_val_42.jpg \
    Global.use_gpu=False

[Errno 2] No such file or directory: 'PaddleOCR/'
/home/nguyen/workplace/signboard_ocr/PaddleOCR
[2024/07/17 11:02:49] ppocr INFO: Architecture : 
[2024/07/17 11:02:49] ppocr INFO:     Backbone : 
[2024/07/17 11:02:49] ppocr INFO:         checkpoints : ../paddleocr_data/pretrained_model/ser_vi_layoutxlm_xfund_pretrained/best_accuracy
[2024/07/17 11:02:49] ppocr INFO:         mode : vi
[2024/07/17 11:02:49] ppocr INFO:         name : LayoutXLMForSer
[2024/07/17 11:02:49] ppocr INFO:         num_classes : 7
[2024/07/17 11:02:49] ppocr INFO:         pretrained : True
[2024/07/17 11:02:49] ppocr INFO:     Transform : None
[2024/07/17 11:02:49] ppocr INFO:     algorithm : LayoutXLM
[2024/07/17 11:02:49] ppocr INFO:     model_type : kie
[2024/07/17 11:02:49] ppocr INFO: Eval : 
[2024/07/17 11:02:49] ppocr INFO:     dataset : 
[2024/07/17 11:02:49] ppocr INFO:         data_dir : train_data/XFUND/zh_val/image
[2024/07/17 11:02:49] ppocr INFO:         label_file_list : ['train_data/XFUND/zh_val

In [28]:
# predict using SER and RE trained model at the same time
%cd /home/nguyen/workplace/signboard_ocr/PaddleOCR
!python3 tools/infer_kie_token_ser_re.py \
    -c configs/kie/vi_layoutxlm/re_vi_layoutxlm_xfund_zh.yml \
    -o Architecture.Backbone.checkpoints=../paddleocr_data/pretrained_model/re_vi_layoutxlm_xfund_pretrained/best_accuracy \
    Global.infer_img=train_data/XFUND/zh_val/image/zh_val_42.jpg \
    Global.use_gpu=False\
    -c_ser configs/kie/vi_layoutxlm/ser_vi_layoutxlm_xfund_zh.yml \
    -o_ser Architecture.Backbone.checkpoints=../paddleocr_data/pretrained_model/ser_vi_layoutxlm_xfund_pretrained/best_accuracy
%cd ..

/home/nguyen/workplace/signboard_ocr/PaddleOCR
[2024/07/17 11:56:07] ppocr INFO: ********** re config **********
[2024/07/17 11:56:07] ppocr INFO: Architecture : 
[2024/07/17 11:56:07] ppocr INFO:     Backbone : 
[2024/07/17 11:56:07] ppocr INFO:         checkpoints : ../paddleocr_data/pretrained_model/re_vi_layoutxlm_xfund_pretrained/best_accuracy
[2024/07/17 11:56:07] ppocr INFO:         mode : vi
[2024/07/17 11:56:07] ppocr INFO:         name : LayoutXLMForRe
[2024/07/17 11:56:07] ppocr INFO:         pretrained : True
[2024/07/17 11:56:07] ppocr INFO:     Transform : None
[2024/07/17 11:56:07] ppocr INFO:     algorithm : LayoutXLM
[2024/07/17 11:56:07] ppocr INFO:     model_type : kie
[2024/07/17 11:56:07] ppocr INFO: Eval : 
[2024/07/17 11:56:07] ppocr INFO:     dataset : 
[2024/07/17 11:56:07] ppocr INFO:         data_dir : train_data/XFUND/zh_val/image
[2024/07/17 11:56:07] ppocr INFO:         label_file_list : ['train_data/XFUND/zh_val/val.json']
[2024/07/17 11:56:07] ppocr INFO